In [1]:
import numpy as np
import random
import re
import time
from threading import Thread, Semaphore

In [4]:
class Puzzle:
    agents = []
    grid = []
    # initialize the board with the size of the puzzle  
    def __init__(self, row_size, fullfilness):
        self.fullfilness = fullfilness/100
        self.row_size = self.col_size = Puzzle.row_size = Puzzle.col_size= row_size
        self.grid_size = self.row_size ** 2
        
        # randomly generate the position of the agents and values
        pos = [(row,col) for row in range(0,self.row_size) for col in range(0,self.col_size)]
        self.max_agents = self.grid_size-1
        self.nbAgents = int(self.max_agents * self.fullfilness)
        
        self.target = random.sample(pos, k=self.nbAgents)
        self.position = random.sample(pos, k=self.nbAgents)
        
        Puzzle.grid = [[None for _ in range(self.col_size)] for _ in range(self.row_size)]
    
    def showGrid():
        print("\n".join([' '.join(list(map(str,x))) for x in Puzzle.grid]))
 
 
               
class Agent(Thread): 
    
    semaphore = Semaphore(1)
    
    def __init__(self, current_postition, target_position):
        Thread.__init__(self)
        self.current_position = current_postition
        self.target_position = target_position
        Puzzle.grid[self.current_position[0]][self.current_position[1]] = self
        Puzzle.agents.append(self)
        self.running = True
        
    def run(self):
        while self.running:
            self.semaphore.acquire()
            self.move_agent()
            self.semaphore.release()
            time.sleep(0.01)
            
    def move_agent(self):
        neighbors = self.get_neighbors()
        
        for neighbor, pos in neighbors:
            if neighbor == None:
                current_position = self.current_position
                self.current_position = pos
                Puzzle.grid[self.current_position[0]][self.current_position[1]] = self
                Puzzle.grid[current_position[0]][current_position[1]] = None  
                return
            

    def get_neighbors(self):
        row= self.current_position[0]
        col= self.current_position[1]
        
        neighbors = []
        if row > 0:
            neighbors.append([Puzzle.grid[row-1][col], (row-1, col)])
        if row < Puzzle.row_size-1:
            neighbors.append([Puzzle.grid[row+1][col], (row+1, col)])
        if col > 0:
            neighbors.append([Puzzle.grid[row][col-1], (row, col-1)])
        if col < Puzzle.col_size-1:
            neighbors.append([Puzzle.grid[row][col+1], (row, col+1)])
        return neighbors      
    

In [5]:
Puzzle.agents

[]

In [7]:
Puzzle.agents = []
Puzzle.grid=[]

p = Puzzle(3, 30)
for target, pos in zip(p.position, p.target):
    Agent(pos, target)
    

for agent in Puzzle.agents:
    agent.start()
    
max_time= 5 # seconds
init_time = time.time()
done = False

while not done:
    time.sleep(1)
    Puzzle.showGrid()
    print("\n")
    
    if time.time() - init_time > max_time:
        done = True
        print("Timeout")

<Agent(Thread-8, started 13239058432)> None <Agent(Thread-5, started 13188689920)>
None <Agent(Thread-6, started 13205479424)> None
<Agent(Thread-7, started 13222268928)> None None


None <Agent(Thread-6, started 13205479424)> None
<Agent(Thread-8, started 13239058432)> None <Agent(Thread-5, started 13188689920)>
None <Agent(Thread-7, started 13222268928)> None


None None None
<Agent(Thread-7, started 13222268928)> <Agent(Thread-6, started 13205479424)> <Agent(Thread-8, started 13239058432)>
None <Agent(Thread-5, started 13188689920)> None


None None <Agent(Thread-5, started 13188689920)>
<Agent(Thread-7, started 13222268928)> <Agent(Thread-8, started 13239058432)> None
None <Agent(Thread-6, started 13205479424)> None


None None <Agent(Thread-6, started 13205479424)>
<Agent(Thread-8, started 13239058432)> <Agent(Thread-7, started 13222268928)> None
<Agent(Thread-5, started 13188689920)> None None


Timeout


In [69]:
Puzzle.grid

[[None, <Agent(Thread-73, initial)>, <Agent(Thread-74, initial)>],
 [None, <Agent(Thread-75, initial)>, None],
 [<Agent(Thread-77, initial)>,
  <Agent(Thread-72, initial)>,
  <Agent(Thread-76, initial)>]]

In [25]:
p.nbAgents, p.max_agents

(6, 8)

In [32]:
class Puzzle:
    # initialize the board with the size of the puzzle  
    def __init__(self, row_size, fullfilness):
        self.fullfilness = fullfilness
        self.row_size = self.col_size = row_size
        self.grid_size = self.row_size ** 2
        
        # randomly generate the position of the agents and values
        pos = [(row,col) for row in range(1,self.row_size+1) for col in range(1,self.col_size+1)]
        target = random.sample(pos, k=len(self.fullfilness * self.grid_size))
        self.value = [i for i in range(1, self.grid_size)] 
        self.pos_row = random.sample(self.value, k=len(self.value))
        self.pos_col= random.sample(self.value, k=len(self.value))
        self.agent = dict({agent: (row, col) for agent,row,col in zip(self.value, self.pos_row, self.pos_col)}) 
        print(self.agent)
        self.agents_temp = self.agents + [None]
        random.shuffle(self.agents_temp)
        self.targets = [self.agents_temp[x:x+ self.row_size] for x in range(0, len(self.agents_temp),self.row_size)]
        
        print(self.targets)
        self.none_row = None
        self.none_column = None
        self.grid = self.agents_temp
        
    # function to randomly put the position of the agents
    def shuffle_agents(self):
        random.shuffle(self.grid)
        self.grid = [self.grid[x:x+ self.row_size] for x in range(0, len(self.agents_temp),self.row_size)]

    def get_neighbors(self, row, col):
        neighbors = []
        if row > 0:
            neighbors.append(self.board[row-1][col])
        if row < self.row_size:
            neighbors.append(self.board[row+1][col])
        if col > 0:
            neighbors.append(self.board[row][col-1])
        if col < self.col_size:
            neighbors.append(self.board[row][col+1])
        return neighbors


    def move_agent(self, piece):
        # move piece to empty space
        row, column = piece.row, piece.col
        empty_row, empty_column = self.none_row, self.none_column
        
        # swap the agent and none agent
        self.grid[row][column], self.grid[empty_row][empty_column] = self.grid[empty_row][empty_column], self.grid[row][column]
        
        # update locations of the agent and none agent
        piece.row, piece.col = empty_row, empty_column
        self.none_row, self.none_column = row, column
      
    # if actual grid is equal to target grid then the puzzle is solved
    def solved(self):
        return self.grid == self.targets
     
    def print_board(self):
        for row in self.grid:
            print([p if p is not None else None for p in row])

    def solve(self):
        threads = []
        for agent in self.agents:
            agent = Agent(agent, self)
            threads.append(agent)
            agent.start()
        
        for thread in threads:
            thread.join()
            
class Agent(Thread):
    
    def __init__(self, agents, puzzle):
        Thread.__init__(self)
        self.agents = agents
        self.puzzle = puzzle
        self.semaphore = Semaphore(1)
        
    def run(self):
        while not self.puzzle.solved():
            neighbors = self.puzzle.get_neighbors(self.agents.row, self.agents.col)
            target = None
            for nei in neighbors:
                if self.agents.value == nei.value + 1:
                    target = nei
                    break
                else: 
                    if nei.value == 1:
                        target = nei
                        break
                
            if target is not None:
                self.semaphore.acquire()
                if self.puzzle.solved():
                    self.semaphore.release()
                    break
                
                if target.row != self.puzzle.empty_row or target.col != self.puzzle.empty_col:
                    self.puzzle.move_agent(target)
                    print(f"Agent {self.agents.name} moved {target.name} to {target.row}, {target.col}")

                self.semaphore.release() 
                
            else:
                time.sleep(0.1)               
        
if __name__ == '__main__':
    row_test = int(input("Enter the size of the puzzle: "))
    fullfilness = int(input("Enter the ratio of fullfilness [0,1]: "))
    
    puzzle = Puzzle(row_test, fullfilness)
    puzzle.shuffle_agents()
    print("Initial Board:")
    puzzle.print_board()
    
    start_time = time.time()
    puzzle.solve()
    end_time = time.time()

    print("Solved Board:")
    puzzle.print_board()
    print("Time taken: {:.4f} seconds".format(end_time - start_time))
        

[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]
{1: (5, 8), 2: (1, 5), 3: (2, 7), 4: (7, 3), 5: (3, 4), 6: (4, 2), 7: (6, 1), 8: (8, 6)}
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/mariapaularoulet/opt/anaconda3/envs/tortilla/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/_d/yrsztpyn10v0f0n2_c361fq80000gn/T/ipykernel_1090/1453857605.py", line 110, in <module>
    puzzle = Puzzle(row_test)
  File "/var/folders/_d/yrsztpyn10v0f0n2_c361fq80000gn/T/ipykernel_1090/1453857605.py", line 15, in __init__
    self.agents_temp = self.agents + [None]
AttributeError: 'Puzzle' object has no attribute 'agents'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/mariapaularoulet/opt/anaconda3/envs/tortilla/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 1997, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/Users/mariapaularoulet/opt/anaconda3/envs/tortilla/lib/python3.8/site-packages/IPython/core/ultrat

In [150]:
random

In [164]:
s = [1,2, 3, 4,5,6]

In [167]:
s.insert(4, None)